In [9]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [11]:
def get_output(message):
    linshi=random.randint(0,3)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: (The option your choice)",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [12]:
def get_prompt(ans,fact):
    template="""Please help me classify the following economic phenomenon.

Economic Phenomenon: %s

A. expansionary economic phenomenon   B. contractionary economic phenomenon

Output format:
Ans: (The category of your choice)"""%(fact)
    return template

In [13]:
def action(prompt,memory):
    template='''Please complete the following tasks based on previous experience.

Previous Experience:[%s]

Task:[%s]
'''%(memory,prompt)
    return template

In [14]:
def reward(prompt,ans):
    template='''Please according task description and answer to determine whether the current answer is correct.

Task:[%s]

Answer:[%s]

Output Format: 
(Right / Wrong)
'''%(prompt,ans)
    return template

In [15]:
def get_exp(prompt,ans):
    template='''The following answer is wrong, please sum up the wrong experience.

[ %s
%s ]


Output Format:
Experience: (Content)
'''%(prompt,ans)
    return template

In [16]:
data= open('../data/test.jsonl','r')

for i in trange(500):
    Experience=''
    question=json.loads(data.readline())
    if i<=460:
        continue
    jishu=0
    prompt=get_prompt(str(question['ans']),question['question'])
    
    while jishu<=3:
        ans=get_output(action(prompt,Experience))
        rew=get_output(reward(prompt,ans))

        if 'Right' in rew or 'right' in rew:
            break
        
        Experience=Experience+'\n'+get_output(get_exp(prompt,ans))
        jishu=jishu+1


    with open("reflexion.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [01:29<00:00,  5.59it/s] 
